# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup as BS
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests # library to handle requests


## 1. Scrapping and preparing neighborhoods data

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = get(url)
soup = BS(response.text, 'html.parser')

In [3]:
my_table = soup.find('table',{'class':'wikitable sortable'})
rows = my_table.find_all('tr')

In [4]:
data = dict()
for index, row in enumerate(rows[1:]):
    data[index] = [c.text.strip() for c in row.find_all('td')]
c_names = [c.text.strip() for c in rows[0].find_all('th')]

In [5]:
toronto_df = pd.DataFrame.from_dict(data, orient='index', columns= c_names)
toronto_df.sample(5)

Postcode       Borough   Neighbourhood
112      M3L    North York  Downsview West
264      M5Y  Not assigned    Not assigned
201      M6T  Not assigned    Not assigned
109      M1L   Scarborough        Oakridge
7        M6A    North York  Lawrence Manor

In [6]:
# Dropping cells with a no assigned borough.
toronto_df = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)

In [7]:
#In the case of cells having a borough but a Not assigned neighborhood, the neighborhood will be the same as the borough
toronto_df.loc[toronto_df.Neighbourhood=="Not assigned",'Neighbourhood'] = toronto_df[toronto_df.Neighbourhood=="Not assigned"].Borough

In [8]:
toronto_df = toronto_df.reset_index(drop=True)
toronto_df.tail()

Postcode    Borough             Neighbourhood
207      M8Z  Etobicoke  Kingsway Park South West
208      M8Z  Etobicoke                 Mimico NW
209      M8Z  Etobicoke        The Queensway West
210      M8Z  Etobicoke     Royal York South West
211      M8Z  Etobicoke            South of Bloor

#### Using geopy library to get the latitude and longitude values of the diffent neighborhoods:

In [9]:
%%time
geolocator = Nominatim(user_agent="toronto_explorer",timeout=10)
def add_coord(i):
    toronto_df.loc[i,'Latitude'] = location.latitude
    toronto_df.loc[i,'Longitude'] = location.longitude
for i in range(len(toronto_df)):
    address = toronto_df.Neighbourhood[i].replace('-',' ')+ ' ' + toronto_df.Borough[i]
    location = geolocator.geocode(address)
    if location:
        add_coord(i)
    else: # if not found, we try Toronto rather than the borough
        address =toronto_df.Neighbourhood[i].replace('-',' ') + ' Toronto'
        location = geolocator.geocode(address)
        if location:
            add_coord(i)
        else: # if not found, we try Toronto rather than the borough
            address = toronto_df.Neighbourhood[i].replace('-',' ')
            location = geolocator.geocode(address)
            if location:
                add_coord(i)

Wall time: 2min 18s


Let's check if there are outliers (not correctly identified addresses):

In [10]:
toronto_df[((toronto_df['Longitude'] < -81) |(toronto_df['Longitude'] > -78)
           |(toronto_df['Latitude'] < 43.5) |(toronto_df['Latitude'] > 45))==True]

Postcode           Borough        Neighbourhood   Latitude   Longitude
6        M7A      Queen's Park         Queen's Park  50.824324   -0.125831
38       M6E              York  Caledonia-Fairbanks  44.420060  -72.019779
102      M6M              York              Del Ray  38.837669  -77.049739
114      M6N              York   The Junction North  42.928672  -78.026122
132      M5R   Central Toronto        North Midtown  43.162128  -77.619604
169      M5V  Downtown Toronto        Railway Lands  49.266347 -123.236746
171      M8V         Etobicoke    Humber Bay Shores  53.521251   -1.136559
175      M9V         Etobicoke     Beaumond Heights  43.130212  -75.200891

Those two coordinates respectively point to Rochester NY and Vancouver BC, we will fix that. Let's check also if there is still some missing coordinates values:

In [11]:
toronto_df[toronto_df.Latitude.isnull()==True]

Postcode           Borough  \
70       M3K        North York   
136      M7R       Mississauga   
185      M5W  Downtown Toronto   
198      M7Y      East Toronto   

                                         Neighbourhood  Latitude  Longitude  
70                                         CFB Toronto       NaN        NaN  
136              Canada Post Gateway Processing Centre       NaN        NaN  
185                    Stn A PO Boxes 25 The Esplanade       NaN        NaN  
198  Business Reply Mail Processing Centre 969 Eastern       NaN        NaN

To fill the missing and incorrect values, I decided to enter those manually since the geopy library was not able to capture them correctly (and it is a small number anyway):

In [12]:
rest_missing = {6:[43.664659, -79.392453], 38:[43.695556, -79.45], 70:[43.743889, -79.465556], 102: [43.713325, -79.428424],
                114:[43.665556, -79.464444], 132: [43.7057358, -79.3975669174092], 136:[43.636745, -79.617856], 169:[43.644662, -79.38077],
                171:[43.626, -79.457], 175:[43.737222, -79.565278], 185:[43.646124, -79.375669], 198: [43.662905, -79.323222] }
for key in rest_missing.keys():
    toronto_df.loc[key,[ 'Latitude','Longitude']] = rest_missing[key]

In [13]:
toronto_df[((toronto_df['Longitude'] < -81) |(toronto_df['Longitude'] > -78)
           |(toronto_df['Latitude'] < 43.5) |(toronto_df['Latitude'] > 45) | toronto_df.Latitude.isnull())==True]

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood, Latitude, Longitude]
Index: []

There is no more outliers or NaN values for coordinates! Next step is to combine neighborhoods having the same postal code

#### Combining neighborhoods having the same postal code:

In [14]:
multi_neighb_postecode = {}
for code in toronto_df.Postcode.unique():
    if len(toronto_df.groupby('Postcode').get_group(code))>1:
        temp = toronto_df.groupby('Postcode').get_group(code).Neighbourhood.values
        multi_neighb_postecode[code] = ''.join([g + ', ' for g in temp[:-1]]+[temp[-1]])
        
for code in multi_neighb_postecode.keys(): 
    for index in toronto_df[toronto_df['Postcode']==code].index:
        toronto_df.loc[index,'Neighbourhood'] = multi_neighb_postecode[code]

toronto_df.drop_duplicates(subset='Postcode', keep='first', inplace=True) # or we can also take the mean of the coordinates
toronto_df = toronto_df.reset_index(drop=True)

In [15]:
toronto_df.sample(5)

Postcode           Borough  \
26      M1H       Scarborough   
52      M2M        North York   
42      M5K  Downtown Toronto   
76      M7R       Mississauga   
51      M1M       Scarborough   

                                      Neighbourhood   Latitude  Longitude  
26                                        Cedarbrae  43.756467 -79.226692  
52                          Newtonbrook, Willowdale  43.793886 -79.425679  
42         Design Exchange, Toronto Dominion Centre  43.647741 -79.380115  
76            Canada Post Gateway Processing Centre  43.636745 -79.617856  
51  Cliffcrest, Cliffside, Scarborough Village West  43.721939 -79.236232

In [16]:
toronto_df.shape

(103, 5)

## 2. Segmenting and Clustering Neighborhoods

In [17]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


##### Create a map of Toronto with neighborhoods superimposed on top.

In [18]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Foursquare Credentials and Version

In [1]:
CLIENT_ID = 'enter_your_CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'enter_your_CLIENT_SECRET' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(3140, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Parkwoods              43.761124              -79.324059   
1    Parkwoods              43.761124              -79.324059   
2    Parkwoods              43.761124              -79.324059   
3    Parkwoods              43.761124              -79.324059   
4    Parkwoods              43.761124              -79.324059   

                Venue  Venue Latitude  Venue Longitude        Venue Category  
0     Allwyn's Bakery       43.759840       -79.324719  Caribbean Restaurant  
1         Tim Hortons       43.760668       -79.326368                  Café  
2          A&W Canada       43.760643       -79.326865  Fast Food Restaurant  
3         Food Basics       43.760865       -79.326015           Supermarket  
4  Shoppers Drug Mart       43.760857       -79.324961              Pharmacy

Let's find out how many unique categories can be curated from all the returned venues

In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


### Neighborhoods Analyzis

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = ['Neighborhood'] + [c for c in toronto_onehot.columns if c!='Neighborhood']
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighborhood  Accessories Store  Afghan Restaurant  American Restaurant  \
0    Parkwoods                  0                  0                    0   
1    Parkwoods                  0                  0                    0   
2    Parkwoods                  0                  0                    0   
3    Parkwoods                  0                  0                    0   
4    Parkwoods                  0                  0                    0   

   Antique Shop  Aquarium  Arepa Restaurant  Argentinian Restaurant  \
0             0         0                 0                       0   
1             0         0                 0                       0   
2             0         0                 0                       0   
3             0         0                 0                       0   
4             0         0                 0                       0   

   Art Gallery  Art Museum     ...       Video Game Store  Video Store  \
0            0           0     ...                      0            0   
1            0           0     ...                      0            0   
2            0           0     ...                      0            0   
3            0           0     ...                      0            0   
4            0           0     ...                      0            0   

   Vietnamese Restaurant  Warehouse Store  Whisky Bar  Wine Bar  Wine Shop  \
0                      0                0           0         0          0   
1                      0                0           0         0          0   
2                      0                0           0         0          0   
3                      0                0           0         0          0   
4                      0                0           0         0          0   

   Wings Joint  Women's Store  Yoga Studio  
0            0              0            0  
1            0              0            0  
2            0              0            0  
3            0              0            0  
4            0              0            0  

[5 rows x 274 columns]

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.sample(3)

Neighborhood  Accessories Store  \
34  Dorset Park, Scarborough Town Centre, Wexford ...                0.0   
29                                   Davisville North                0.0   
60                      L'Amoreaux West, Steeles West                0.0   

    Afghan Restaurant  American Restaurant  Antique Shop  Aquarium  \
34                0.0                  0.0           0.0       0.0   
29                0.0                  0.0           0.0       0.0   
60                0.0                  0.0           0.0       0.0   

    Arepa Restaurant  Argentinian Restaurant  Art Gallery  Art Museum  \
34               0.0                     0.0          0.0         0.0   
29               0.0                     0.0          0.0         0.0   
60               0.0                     0.0          0.0         0.0   

       ...       Video Game Store  Video Store  Vietnamese Restaurant  \
34     ...                    0.0          0.0                    0.0   
29     ...                    0.0          0.0                    0.0   
60     ...                    0.0          0.0                    0.0   

    Warehouse Store  Whisky Bar  Wine Bar  Wine Shop  Wings Joint  \
34              0.0         0.0       0.0        0.0     0.090909   
29              0.0         0.0       0.0        0.0     0.000000   
60              0.0         0.0       0.0        0.0     0.000000   

    Women's Store  Yoga Studio  
34            0.0          0.0  
29            0.0          0.0  
60            0.0          0.0  

[3 rows x 274 columns]

Let's put that into a pandas dataframe and display the top 10 venues for each neighborhood.

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                          Agincourt    Chinese Restaurant   
2  Agincourt North, L'Amoreaux East, Milliken, St...    Chinese Restaurant   
3  Albion Gardens, Beaumond Heights, Humbergate, ...                  Bank   
4                             Alderwood, Long Branch           Pizza Place   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0     Italian Restaurant                   Bar        Breakfast Spot   
1  Vietnamese Restaurant  Cantonese Restaurant           Coffee Shop   
2                 Bakery  Fast Food Restaurant        Ice Cream Shop   
3               Pharmacy         Grocery Store          Liquor Store   
4                   Pool                   Gym           Coffee Shop   

  5th Most Common Venue   6th Most Common Venue 7th Most Common Venue  \
0               Theater  Furniture / Home Store           Karaoke Bar   
1         Train Station        Asian Restaurant     Korean Restaurant   
2   Fried Chicken Joint                    Bank             Gift Shop   
3            Beer Store             Gas Station           Flower Shop   
4                   Pub            Skating Rink          Dance Studio   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                   Pub      Sushi Restaurant            Event Space  
1  Hong Kong Restaurant           Fish Market      Fish & Chips Shop  
2             Juice Bar          Liquor Store            Coffee Shop  
3           Flea Market           Fish Market                Exhibit  
4              Pharmacy        Sandwich Place      Fish & Chips Shop

## Clustering Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [62]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 1, 3, 1, 2])

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df
toronto_merged.columns = list(toronto_merged.columns[:2]) + ['Neighborhood'] +  list(toronto_merged.columns[3:])

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged['Cluster Labels'] = np.nan_to_num(toronto_merged['Cluster Labels']).astype(int)
toronto_merged.head() # check the last columns!

Postcode           Borough                      Neighborhood   Latitude  \
0      M3A        North York                         Parkwoods  43.761124   
1      M4A        North York                  Victoria Village  43.732658   
2      M5A  Downtown Toronto         Harbourfront, Regent Park  43.640080   
3      M6A        North York  Lawrence Heights, Lawrence Manor  43.722778   
4      M7A      Queen's Park                      Queen's Park  43.664659   

   Longitude  Cluster Labels      1st Most Common Venue 2nd Most Common Venue  \
0 -79.324059             1.0          Convenience Store       Laundry Service   
1 -79.311189             2.0  Middle Eastern Restaurant                  Park   
2 -79.380150             1.0                Coffee Shop                  Café   
3 -79.450933             1.0             Clothing Store   Sporting Goods Shop   
4 -79.392453             1.0                Coffee Shop                  Café   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0           Pizza Place        Discount Store           Coffee Shop   
1      Asian Restaurant       Thai Restaurant               Exhibit   
2                 Hotel               Brewery           Pizza Place   
3   American Restaurant           Coffee Shop      Toy / Game Store   
4                  Park        Sandwich Place        Clothing Store   

  6th Most Common Venue 7th Most Common Venue   8th Most Common Venue  \
0    Chinese Restaurant  Caribbean Restaurant                    Café   
1    Falafel Restaurant        Farmers Market    Fast Food Restaurant   
2            Restaurant         Boat or Ferry             Music Venue   
3        Cosmetics Shop            Food Court  Furniture / Home Store   
4      Sushi Restaurant                   Gym             Pizza Place   

  9th Most Common Venue 10th Most Common Venue  
0  Fast Food Restaurant          Shopping Mall  
1                 Field    Filipino Restaurant  
2                  Park     Italian Restaurant  
3        Sandwich Place          Jewelry Store  
4                Museum                    Pub

Finally, let's visualize the resulting clusters

In [75]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
17    Etobicoke               0     Convenience Store        Sandwich Place   
23    East York               0        Sandwich Place     Convenience Store   
28   North York               0            Playground     Convenience Store   
38  Scarborough               0         Grocery Store     Convenience Store   
82  Scarborough               0  Caribbean Restaurant     Convenience Store   
88    Etobicoke               0                   NaN                   NaN   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
17          Intersection            Donut Shop        Shipping Store   
23   Japanese Restaurant                  Park                 Field   
28                  Park        Baseball Field                  Food   
38            Beer Store           Gas Station     Food & Drink Shop   
82     Fish & Chips Shop               Exhibit    Falafel Restaurant   
88                   NaN                   NaN                   NaN   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
17           Coffee Shop                 Field               Exhibit   
23           Event Space               Exhibit    Falafel Restaurant   
28           Flower Shop           Flea Market           Fish Market   
38                  Food           Flower Shop           Flea Market   
82        Farmers Market  Fast Food Restaurant                 Field   
88                   NaN                   NaN                   NaN   

   9th Most Common Venue 10th Most Common Venue  
17    Falafel Restaurant         Farmers Market  
23        Farmers Market   Fast Food Restaurant  
28     Fish & Chips Shop   Ethiopian Restaurant  
38           Fish Market            Event Space  
82   Filipino Restaurant            Yoga Studio  
88                   NaN                    NaN

#### Cluster 2

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels       1st Most Common Venue  \
0          North York               1           Convenience Store   
2    Downtown Toronto               1                 Coffee Shop   
3          North York               1              Clothing Store   
4        Queen's Park               1                 Coffee Shop   
5           Etobicoke               1                 Coffee Shop   
7          North York               1         American Restaurant   
8           East York               1                 Coffee Shop   
9    Downtown Toronto               1                 Coffee Shop   
10         North York               1                  Playground   
11          Etobicoke               1        Fast Food Restaurant   
12        Scarborough               1  Construction & Landscaping   
13         North York               1               Deli / Bodega   
14          East York               1                Skating Rink   
15   Downtown Toronto               1                 Coffee Shop   
16               York               1           Convenience Store   
18        Scarborough               1               Train Station   
19       East Toronto               1                       Beach   
20   Downtown Toronto               1                 Coffee Shop   
21               York               1                        Bank   
22        Scarborough               1        Fast Food Restaurant   
24   Downtown Toronto               1                 Coffee Shop   
25   Downtown Toronto               1           Korean Restaurant   
26        Scarborough               1        Fast Food Restaurant   
27         North York               1           Korean Restaurant   
29          East York               1           Afghan Restaurant   
30   Downtown Toronto               1                 Coffee Shop   
31       West Toronto               1                        Café   
32        Scarborough               1                 Coffee Shop   
33         North York               1            Ramen Restaurant   
35          East York               1                        Park   
..                ...             ...                         ...   
64               York               1               Train Station   
65        Scarborough               1           Indian Restaurant   
66         North York               1                   Wine Shop   
67    Central Toronto               1                 Pizza Place   
71        Scarborough               1                 Coffee Shop   
72         North York               1                 Coffee Shop   
73    Central Toronto               1                 Art Gallery   
74    Central Toronto               1                 Pizza Place   
75       West Toronto               1                        Café   
76        Mississauga               1                 Coffee Shop   
78        Scarborough               1          Chinese Restaurant   
79    Central Toronto               1                 Pizza Place   
80   Downtown Toronto               1           Korean Restaurant   
81       West Toronto               1                 Coffee Shop   
84   Downtown Toronto               1                        Café   
85        Scarborough               1          Chinese Restaurant   
86    Central Toronto               1                 Coffee Shop   
87   Downtown Toronto               1                       Hotel   
89          Etobicoke               1                        Bank   
90        Scarborough               1          Chinese Restaurant   
91   Downtown Toronto               1              Clothing Store   
92   Downtown Toronto               1                 Coffee Shop   
93          Etobicoke               1                 Pizza Place   
94          Etobicoke               1                         Gym   
96   Downtown Toronto               1                 Coffee Shop   
97   Downtown Toronto               1                 Coffee Shop   
98          

#### Cluster 3

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
1         North York               2  Middle Eastern Restaurant   
6        Scarborough               2       Fast Food Restaurant   
34        North York               2                       Park   
45        North York               2                       Park   
49        North York               2                     Bakery   
50        North York               2                     Bakery   
58       Scarborough               2                       Park   
68   Central Toronto               2                 Playground   
69      West Toronto               2                       Park   
70         Etobicoke               2                Pizza Place   
77         Etobicoke               2     Furniture / Home Store   
83   Central Toronto               2                 Playground   
95       Scarborough               2       Fast Food Restaurant   
101        Etobicoke               2              Deli / Bodega   

          2nd Most Common Venue     3rd Most Common Venue  \
1                          Park          Asian Restaurant   
6                      Pharmacy                      Park   
34                   Food Truck            Baseball Field   
45    Middle Eastern Restaurant              Tennis Court   
49   Construction & Landscaping                      Park   
50   Construction & Landscaping       Empanada Restaurant   
58                          Gym                  Gym Pool   
68                         Park  Mediterranean Restaurant   
69           Light Rail Station      Gym / Fitness Center   
70                         Park               Yoga Studio   
77                         Park               Yoga Studio   
83                          Gym              Tennis Court   
95                     Pharmacy                      Park   
101  Construction & Landscaping                      Park   

    4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
1         Thai Restaurant               Exhibit    Falafel Restaurant   
6             Yoga Studio   Filipino Restaurant               Exhibit   
34            Yoga Studio   Filipino Restaurant    Falafel Restaurant   
45                Exhibit    Falafel Restaurant        Farmers Market   
49       Business Service                  Food     Food & Drink Shop   
50                   Park                   Gym                  Food   
58   Ethiopian Restaurant               Exhibit    Falafel Restaurant   
68                   Bank               Exhibit    Falafel Restaurant   
69                 Bridge   American Restaurant                 Beach   
70    Filipino Restaurant               Exhibit    Falafel Restaurant   
77      Fish & Chips Shop    Falafel Restaurant        Farmers Market   
83                   Park  Ethiopian Restaurant               Exhibit   
95            Yoga Studio   Filipino Restaurant               Exhibit   
101           Yoga Studio     Fish & Chips Shop    Falafel Restaurant   

    7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
1          Farmers Market  Fast Food Restaurant                 Field   
6      Falafel Restaurant        Farmers Market                 Field   
34         Farmers Market  Fast Food Restaurant                 Field   
45   Fast Food Restaurant                 Field   Filipino Restaurant   
49            Flower Shop           Flea Market           Fish Market   
50            Flower Shop           Flea Market           Fish Market   
58         Farmers Market  Fast Food Restaurant                 Field   
68         Farmers Market  Fast Food Restaurant                 Field   
69               Building              Bus Stop                 Trail   
70         Farmers Market  Fast Food Restaurant                 Field   
77   Fast Food Restaurant                 Field   Filipino Restaurant   
83     Falafel Restaurant        Farmers Market  Fast Food Restaurant   
95     Falafel Restaurant        Farmers Marke

#### Cluster 4

In [79]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
55  North York               3             Locksmith   Rental Car Location   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
55           Yoga Studio  Ethiopian Restaurant               Exhibit   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
55    Falafel Restaurant        Farmers Market  Fast Food Restaurant   

   9th Most Common Venue 10th Most Common Venue  
55                 Field    Filipino Restaurant

#### Cluster 5

In [80]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
40  North York               4          Soccer Field           Yoga Studio   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
40     Fish & Chips Shop               Exhibit    Falafel Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
40        Farmers Market  Fast Food Restaurant                 Field   

   9th Most Common Venue 10th Most Common Venue  
40   Filipino Restaurant            Fish Market